In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

### Brownian motion in $\mathbb{R}^2$

(from last week)

In [ ]:
# sample the SDE using Euler-Maruyama scheme
def sample(dim=2, T=1.0, N=10000, seed=42):
    rng = np.random.default_rng(seed=seed)
    X = [0, 0]
    traj = [X]
    delta_t = T / N
    save = 10
    for i in range(N):
        b = rng.normal(size=(dim,))
        X = X + np.sqrt(delta_t) * b
        if i % save==0:
            traj.append(X)
    return np.array(traj)

trajectory = sample(T=1,seed=42)
plt.plot(trajectory[:,0], trajectory[:,1])
plt.xlim([-2,2])
plt.ylim([-2,2])

### Brownian dynamics 

$$dX_t = -\nabla V(X_t) dt + \sqrt{2\beta^{-1}} dB_t$$

with potential

$$V(x) = (x_1^2-1)^2 + 2.0 * (x_1^2+x_2-1)^2$$ 

first, define $V$ and its gradient

In [ ]:
# a potential function
def V(X):
    return (X[0]**2 - 1)**2 + 2.0 * (X[0]**2 + X[1] - 1)**2

# gradient of potential function 
def gradV(X):
    return np.array(( 4.0 * X[0] * (X[0]**2 - 1.0 + 2.0*(X[0]**2 + X[1] - 1)), 4.0 * (X[0]**2 + X[1] - 1)) )

show the potential profile

In [ ]:
x = np.arange(-2.5, 2.5, 0.05)
y = np.arange(-2.5, 2.5, 0.05)
X, Y = np.meshgrid(x, y)

plt.figure(figsize = (6, 4))

contour_levels = [0.0, 1.0, 1.5, 2.0, 3.0, 4.0]

# evaluate potential on mesh
V_on_grid = V([X,Y])

fig = plt.figure(figsize=(7,4))
ax = fig.add_subplot(1, 1, 1)

# plot profile by pcolormesh
im = ax.pcolormesh(X, Y, V_on_grid, cmap='coolwarm',shading='auto', vmin=0, vmax=4)

# show contour lines
contours = ax.contour(X, Y, V_on_grid, contour_levels)
ax.clabel(contours, inline=True, fontsize=13,colors='black')

ax.set_aspect('equal')
ax.set_xlabel(r'$x_1$',fontsize=20)
ax.set_ylabel(r'$x_2$',fontsize=20, rotation=0)
ax.tick_params(axis='both', labelsize=20)

ax.set_xticks([-2.0, -1.0, 0, 1.0, 2.0])
ax.set_yticks([-2.0, -1.0, 0, 1.0, 2.0])
ax.set_xlim([-2.5, 2.5])
ax.set_ylim([-2.5, 2.5])

ax.set_title('V',fontsize=25)

# show colorbar
cbar = fig.colorbar(im, ax=ax, shrink=1.0)
cbar.ax.tick_params(labelsize=15)
plt.show()

### compute mean using running average

Invariant density:
$$ \pi(x) = \frac{1}{Z} \mathrm{e}^{-\beta V(x)}$$

Mean of a function $g$:

$$ \mathbb{E}_\pi(g) = \frac{1}{Z} \int_{\mathbb{R}^2} g(x) \mathrm{e}^{-\beta V(x)} dx$$


We take $g(x_1,x_2)=x_1$ and estimate the mean using:

$$\mathbb{E}_\pi(g) \approx \frac{1}{N} \sum_{n=1}^N g(x_n)$$

where $x_n$ are sampled from $$dX_t = -\nabla V(X_t) dt + \sqrt{2\beta^{-1}} dB_t$$


In [ ]:

def g(X):
    return X[0]

# sample the SDE using Euler-Maruyama scheme
def sample(beta=1.0, N=10000, seed=42):
    rng = np.random.default_rng(seed=seed)
    X = [-1, 0]
    dim = 2 
    traj = [X]
    delta_t = 0.001
    save = 10
    g_sum = 0.0
    for i in range(N):
        b = rng.normal(size=(dim,))
        X = X - gradV(X) * delta_t + np.sqrt(2 * delta_t/beta) * b
        g_sum = g_sum + g(X) 
        if i % save==0:
            traj.append(X)

    return np.array(traj), g_sum / N

seed_list = [1, 199, 235, 37, 42]
# for each seed, generate a long trajectory 
for seed in seed_list:
    trajectory, g_mean = sample(beta=1.0, N=200000, seed=seed)
    print (r'seed=%d, mean of g: %.4f' % (seed, g_mean))
    
print ("\n\nshape of trajectory array:", trajectory.shape)
# compute values of potential along the trajectory
v_traj = np.array([V(x) for x in trajectory])

### plot 

In [ ]:
fig, ax=plt.subplots(1,2,figsize=(12,5))

ret = ax[0].scatter(trajectory[:,0], trajectory[:,1], c=v_traj)

cbar = plt.colorbar(ret, ax=ax[0], shrink=1.0)
cbar.ax.tick_params(labelsize=15)
ax[0].set_xlabel(r'$x_1$',fontsize=20)
ax[0].set_ylabel(r'$x_2$',fontsize=20, rotation=0)
ax[0].tick_params(axis='both', labelsize=20)
ax[0].set_title('trajectory data',fontsize=25)

ax[1].plot(trajectory[:,0])
ax[1].set_xlabel(r'step',fontsize=20)
ax[1].set_ylabel(r'$x_1$',fontsize=20)

plt.show()

### Exit time

Domain:
$D = \{x=(x_1,x_2)\in \mathbb{R}^2|x_1 < 0.1\}$.

First exit time:
$\tau = \inf_{t>0}\{X_t \not\in D | X_0=(-1,0)\}$.

In [ ]:
# check if still in domain
def in_D(X):
    return X[0]<0.1

# sample the SDE using Euler-Maruyama scheme
def sample(beta=1.0, seed=42):
    rng = np.random.default_rng(seed=seed)
    X = [-1, 0]
    dim = 2 
    traj = [X]
    delta_t = 0.001
    save = 10
    i = 0 
    while True :
        b = rng.normal(size=(dim,))
        X = X - gradV(X) * delta_t + np.sqrt(2 * delta_t/beta) * b
        if not in_D(X) :
            break
        i = i + 1
        if i % save==0:
            traj.append(X)

    return np.array(traj), i * delta_t

seed_list = [1, 199, 235, 37, 42]
tau_list = []
# for each seed, generate a long trajectory 
for seed in seed_list:
    trajectory, tau = sample(beta=2.0, seed=seed)
    tau_list.append(tau)
    #print (r'seed=%d, tau=%.2f' % (seed, tau))

tau_list = np.array(tau_list)
print ('tau_list:', tau_list)
print ('[mean, var]: %.3f, %.3f' % (tau_list.mean(), tau_list.var()))

### OU Process

Brownian dynamics:
$$dX_t = -\nabla V(X_t) dt + \sqrt{2\beta^{-1}} dB_t$$

Invariant density:
$$ \pi(x) = \frac{1}{Z} \mathrm{e}^{-\beta V(x)}$$

With the choice $V(x) = \frac{\kappa |x|^2}{2}$, we get the OU process:

$$dX_t = -\kappa X_t dt + \sqrt{2\beta^{-1}} dB_t$$

The invariant density is (Gaussian):
$$ \pi(x) = \Big(\frac{2\pi}{\beta\kappa}\Big)^{-\frac{d}{2}} \exp\Big(-\frac{\beta\kappa |x|^2}{2}\Big)$$. 

In [ ]:
# sample the SDE using Euler-Maruyama scheme
def sample(dim=1, beta=1.0, kappa=1.0, N=10000, seed=42):
    rng = np.random.default_rng(seed=seed)
    X = np.zeros(dim)
    traj = [X]
    delta_t = 0.001
    save = 5
    for i in range(N):
        b = rng.normal(size=(dim,))
        X = X - kappa * X * delta_t + np.sqrt(2 * delta_t/beta) * b
        if i % save==0:
            traj.append(X)

    return np.array(traj)

beta = 1.0
kappa = 1.0
# generate a long trajectory 
trajectory = sample(dim=1, beta=beta, kappa=kappa, N=1000000, seed=400)
   
print ("Number of states:", trajectory.shape[0])

# plot histogram. The first 40000 states are discarded  
count, bins, _ = plt.hist(trajectory[40000:], 50, density=True) 
# compute invariant density (gaussian) 
gaussian_pdf = 1.0/np.sqrt(2 * np.pi / (beta * kappa)) * np.exp(-beta * kappa * bins**2 / 2)

plt.plot(bins, gaussian_pdf, linewidth=2, color='r')

plt.show()

OU process in $\mathbb{R}^2$

In [ ]:
# generate a long trajectory, set dim=2
trajectory = sample(dim=2, beta=beta, kappa=kappa, N=1000000, seed=400)
print ("Number of states:", trajectory.shape[0])

fig = plt.figure(figsize=(7,5))
ax = fig.add_subplot(1, 1, 1)

# compute histogram statistics
h, xedges, yedges = np.histogram2d(trajectory[:,0], trajectory[:,1], bins=[100, 100], range=[[-2.5,2.5],[-2.5,2.5]], density=True)

# get the meshgrid
X, Y = np.meshgrid(xedges, yedges)
# plot the histogram, specify the colormap and log scale
im = ax.pcolormesh(X, Y, h.T, cmap='coolwarm', shading='auto')

# show colorbar
cbar = fig.colorbar(im, ax=ax, shrink=1.0)
cbar.ax.tick_params(labelsize=15)

ax.set_aspect('equal')
ax.set_xlim([-2.5, 2.5])
ax.set_ylim([-2.5, 2.5])
ax.set_xlabel(r'$x_1$',fontsize=20)
ax.set_ylabel(r'$x_2$',fontsize=20, rotation=0)
ax.tick_params(axis='both', labelsize=20)
ax.set_xticks([-2.0, -1.0, 0, 1.0, 2.0])
ax.set_yticks([-2.0, -1.0, 0, 1.0, 2.0])
plt.show()